In [48]:
import nltk
import pandas as pd
import numpy as np
import sys
from mlxtend.frequent_patterns import apriori, association_rules


ids = pd.read_csv('car_model_identifier.csv', header=0)
ids = ids.sort_values(by=['brand_id'])

brand_ids = ids[['brand','brand_id']]
brand_ids = brand_ids.drop_duplicates().rename(columns={"brand": "brand", "brand_id": "cur_brand_id"})

model_ids = ids[['model_fromsales','model_id']]
model_ids = model_ids.drop_duplicates().rename(columns={"model_fromsales": "model_fromsales", "model_id": "cur_model_id"})


review_info = pd.read_csv('review_brand_model.csv',header = 0)


import re
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()


In [11]:
mentions = pd.read_csv('https://www.dropbox.com/s/plmyo9svj04mai3/entire_data_Mentioned_4th.csv?dl=1', header=0).iloc[:, 1:]
year = pd.read_csv('https://www.dropbox.com/s/888wxzb253wnc5z/org_id_year.csv?dl=1', header=0)
mentions=year.merge(mentions, how='inner', on='org_id')

In [12]:
unique_brand=[]
for x in ids.brand:
    if x not in unique_brand:
         unique_brand.append(x)
            
unique_model=[]
for x in ids['model_fromsales']:
    if x not in unique_model:
         unique_model.append(x)

In [13]:
from itertools import combinations

class lift_calculator:
    def __init__(self, data):
        self.data = data
        self.brand_count = dict(data.sum(axis=0))
        self.pair_brand_count = dict()
        for brand1, brand2 in combinations(data.columns, 2):
            self.pair_brand_count[(brand1, brand2)] = (data[brand1] * data[brand2]).sum()
        self.lift_dict = dict()
        self.total_size = len(data)
        
    def calculate(self):
        for brand1, brand2 in self.pair_brand_count:
            self.lift_dict[(brand1, brand2)] = self.pair_brand_count[(brand1, brand2)] * self.total_size / (self.brand_count[brand1] * self.brand_count[brand2])
    
    def top_n(self, n=10):
        data = pd.DataFrame(self.lift_dict.items()).fillna(0).sort_values(by=1, ascending=False).iloc[:n, :]
        top_n_pairs = dict(zip(data[0], data[1]))
        return top_n_pairs
    
    def return_csv(self):
        df = pd.DataFrame(index=data.columns, columns=data.columns)
        for brand1 in data.columns:
            for brand2 in data.columns:
                df.loc[brand1, brand2] = self.lift_dict.get((brand1, brand2), 0)
        return df

In [68]:
data=mentions.loc[mentions['review_year'] == 2004] ##--- change it back to year
data=data.reset_index(drop=True)
data=pd.concat([data,pd.DataFrame(columns=unique_brand)])
data = data.replace(np.nan, 0)
data.model_id = data.model_id.astype(int)
data = pd.merge(review_info, data, on=['org_id'])
data = pd.merge(data, brand_ids, on = ['brand'])

for i in range(len(data)):
    cur_brand_id = data.iloc[i][50] ##-- (length of brand_ids + 10)
    data.iloc[i,cur_brand_id + 9] = 1
    for y in range (40): ## -- length of brand_ids
        if data.brand_id[i] == y+1:
              data.iloc[i,y+10] += 1 
            
data=data.drop(columns=['review_year','detected_entity','year','model_id','brand_id','yb_id','ybm_id','brand','model_fromsales','cur_brand_id'])
data=data.groupby(['org_id']).sum()

data = data.apply(lambda x: [y if y <= 1 else 1 for y in x])
calculator = lift_calculator(data)
calculator.calculate()
lift_model_df = calculator.return_csv()
lift_model_df.to_csv(str(2004)+"lift_brand.csv2.2") ##--- change it back to year
frequent_itemsets_ap = apriori(data,min_support=0.001, use_colnames=True) ## the min_support could be modified
rules_ap = association_rules(frequent_itemsets_ap, metric="lift", min_threshold=1)
rules_ap.to_csv(str(2004)+"rules_ap_brand.csv2.2") ##--- change it back to year

In [69]:
data

,ACURA,AUDI,BMW,BUICK,CADILLAC,CHEVROLET,CHRYSLER,DAEWOO,DODGE,FORD,...,PORSCHE,RAM,SAAB,SATURN,SCION,SUBARU,SUZUKI,TOYOTA,VOLKSWAGEN,VOLVO
org_id,,,,,,,,,,,,,,,,,,,,,
2102,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2104,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2105,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2106,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2109,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
309828,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
309830,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [55]:
data.iloc[1][10]

1

In [64]:
lift_model_df

,ACURA,AUDI,BMW,BUICK,CADILLAC,CHEVROLET,CHRYSLER,DAEWOO,DODGE,FORD,...,PORSCHE,RAM,SAAB,SATURN,SCION,SUBARU,SUZUKI,TOYOTA,VOLKSWAGEN,VOLVO
ACURA,0,2.00945,1.673,0.306657,0.535903,0.163886,0.0977227,0,0.111574,0.323939,...,0.735076,0,1.77725,0.0561001,1.05789,0.893732,0.136571,0.456352,1.49155,2.06428
AUDI,0,0,3.821,0.15564,0.593435,0.277261,0.148794,0,0.056628,0.300229,...,2.61155,0,4.56648,0.170837,0,1.26361,0,0.280896,2.6588,3.29278
BMW,0,0,0,0.190404,1.02848,0.203514,0.424733,0,0.184737,0.288584,...,2.73846,0.0594557,3.03462,0.13933,0.164211,0.713468,0.254392,0.307465,1.55857,2.6855
BUICK,0,0,0,0,1.81125,1.08319,1.08994,0,0.691348,0.471262,...,0,0.712009,0.412965,0.834273,0.983251,0,0,0.64977,0,0.730912
CADILLAC,0,0,0,0,0,1.16158,0.432894,0,0.197701,0.399302,...,1.15778,0.169675,0.393645,0.298216,0,0.113116,0.241995,0.292481,0.386769,0.435448
CHEVROLET,0,0,0,0,0,0,0.614852,8.82798,1.35474,1.09137,...,0.270465,1.39523,0.588532,0.705941,0.350317,0.549635,0.452253,0.765246,0.216844,0.358067
CHRYSLER,0,0,0,0,0,0,0,0,2.37937,0.55065,...,0.290294,1.02103,0.1974,0.0996969,0,0.226896,0,0.621188,0.323253,0.52407
DAEWOO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,33.1049,1.17681,0,0
DODGE,0,0,0,0,0,0,0,0,0,2.05756,...,0.441921,14.2481,0,0.531197,1.43098,0.604466,0,0.998184,0.147628,0.0664836
FORD,0,0,0,0,0,0,0,0,0,0,...,0.334709,2.1583,0.0569005,0.603491,0.406432,0.654029,0.629637,0.88534,0.391346,0.402836


In [70]:
rules_ap.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(AUDI),(ACURA),0.039418,0.060018,0.004754,0.120603,2.009453,0.002388,1.068894
1,(ACURA),(AUDI),0.060018,0.039418,0.004754,0.079208,2.009453,0.002388,1.043213
2,(ACURA),(BMW),0.060018,0.096662,0.009706,0.161716,1.673000,0.003904,1.077604
3,(BMW),(ACURA),0.096662,0.060018,0.009706,0.100410,1.673000,0.003904,1.044900
4,(ACURA),(HONDA),0.060018,0.185005,0.014559,0.242574,1.311174,0.003455,1.076006
5,(HONDA),(ACURA),0.185005,0.060018,0.014559,0.078694,1.311174,0.003455,1.020271
6,(ACURA),(INFINITI),0.060018,0.038427,0.005744,0.095710,2.490669,0.003438,1.063345
7,(INFINITI),(ACURA),0.038427,0.060018,0.005744,0.149485,2.490669,0.003438,1.105191
8,(ACURA),(LEXUS),0.060018,0.072200,0.008220,0.136964,1.897013,0.003887,1.075042
9,(LEXUS),(ACURA),0.072200,0.060018,0.008220,0.113855,1.897013,0.003887,1.060754
